In [1]:
import functools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Geographic Relationships

In [2]:
# Clean up the file generated from QGIS spatial join
geo = pd.read_csv('./data/TIGER/msa_puma_tract_join.csv', usecols=['jmsa_GEOID','jpuma_GEOI','GEOID']).dropna()
geo['REGION'] = '1'
geo = geo[['REGION','jmsa_GEOID','jpuma_GEOI','GEOID']].astype(int)
geo.columns = ['REGION','MSA','PUMA','TRACT']

# Get rid of Micropolitan Statistical Areas
msa_ids = pd.read_csv('./data/TIGER/msa_list.csv')
geo = geo[geo['MSA'].isin(msa_ids['GEOID'])]
full_msa_list = pd.unique(geo['MSA'])

# Limit to certain MSAs if desired, there are 925 total MSA + MicroSA, 384 MSA
msas_to_use = full_msa_list[350:384]
geo = geo[geo['MSA'].isin(msas_to_use)]

# Remove select problem PUMAs not Matched to Tracts/MSAs in ACS Tables
geo = geo[geo['PUMA']!=2101600]
geo = geo[geo['PUMA']!=4701400]
geo = geo[geo['PUMA']!=4501300]
geo = geo[geo['PUMA']!=5151145]

geo.to_csv('./populationsim-master/example_msa/data/geo_cross_walk.csv', index=False)

# Get the list of pumas, msas, and tracts that are being synthesized
puma_list = list(pd.unique(geo['PUMA']))
puma_list = [int(i) for i in puma_list]
msa_list = list(pd.unique(geo['MSA']))
msa_list = [int(i) for i in msa_list]
tract_list = list(pd.unique(geo['TRACT']))
tract_list = [int(i) for i in tract_list]

## Seed Samples from PUMS

#### Persons

In [3]:
p_data = []
pums_file_list=['./data/PUMS/psam_pusa.csv','./data/PUMS/psam_pusb.csv','./data/PUMS/psam_pusc.csv','./data/PUMS/psam_pusd.csv']
for file in pums_file_list:
    print(file)
    p_df = pd.read_csv(file,
                       usecols=['SERIALNO','ST','PUMA','PWGTP','AGEP','SEX','RAC1P','COW','PINCP','SCHL'],
                       chunksize=100000,
                       dtype=str)
    chunk_list = []
    for chunk in p_df:
        chunk.dropna(inplace=True) # Side effect limits to >16yrs old
        chunk['ST'] = chunk['ST'].astype(str)
        chunk['PUMA'] = chunk['PUMA'].astype(str)
        chunk['PUMA'] = chunk['PUMA'].str.zfill(5)
        chunk['PUMA'] = chunk['ST'] + chunk['PUMA']
        chunk['PUMA'] = chunk['PUMA'].astype(int)
        chunk = chunk[chunk['PUMA'].isin(puma_list)]
        chunk_list.append(chunk)
    p_df = pd.concat(chunk_list)
    p_data.append(p_df)

./data/PUMS/psam_pusa.csv
./data/PUMS/psam_pusb.csv
./data/PUMS/psam_pusc.csv
./data/PUMS/psam_pusd.csv


In [4]:
p_data = pd.concat(p_data)
p_data

,SERIALNO,PUMA,ST,PWGTP,AGEP,COW,SCHL,SEX,PINCP,RAC1P
2646619,2015000002260,4953001,49,6,69,NaN,19,2,19800,1
2646720,2015000007254,4953001,49,33,49,7,16,2,10000,1
2646721,2015000007254,4953001,49,41,51,1,18,1,26100,1
2646900,2015000017602,4953001,49,41,56,1,16,2,38000,1
2646901,2015000017602,4953001,49,50,56,1,16,1,60000,1
...,...,...,...,...,...,...,...,...,...,...
4041701,2019HU1410371,5600400,56,7,6,NaN,03,2,NaN,1
4041702,2019HU1410371,5600400,56,3,5,NaN,02,1,NaN,1
4041703,2019HU1410371,5600400,56,4,2,NaN,NaN,1,NaN,1
4041704,2019HU1410371,5600400,56,4,1,NaN,NaN,1,NaN,1


In [7]:
# Check for NA values
p_data['SCHL'] = p_data['SCHL'].fillna(value=-1) # 1
p_data['PINCP'] = p_data['PINCP'].fillna(value=-10000) # 0
p_data['COW'] = p_data['COW'].fillna(value=-1) # 9
p_data.isna().sum()

SERIALNO    0
PUMA        0
ST          0
PWGTP       0
AGEP        0
COW         0
SCHL        0
SEX         0
PINCP       0
RAC1P       0
dtype: int64

#### Households

In [17]:
h_data = []
pums_file_list=['./data/PUMS/psam_husa.csv','./data/PUMS/psam_husb.csv','./data/PUMS/psam_husc.csv','./data/PUMS/psam_husd.csv']
for file in pums_file_list:
    print(file)
    h_df = pd.read_csv(file,
                       usecols=['SERIALNO','ST','PUMA','WGTP','NP','VEH'],
                       chunksize=100000,
                       dtype=str)
    chunk_list = []
    for chunk in h_df:
        chunk.dropna(inplace=True) # Side effect limits to >16yrs old
        chunk = chunk[chunk['NP'].astype(int) > 0] # Remove households with 0 people in them
        chunk['ST'] = chunk['ST'].astype(str)
        chunk['PUMA'] = chunk['PUMA'].astype(str)
        chunk['PUMA'] = chunk['PUMA'].str.zfill(5)
        chunk['PUMA'] = chunk['ST'] + chunk['PUMA']
        chunk['PUMA'] = chunk['PUMA'].astype(int)
        chunk = chunk[chunk['PUMA'].isin(puma_list)]
        chunk_list.append(chunk)
    h_df = pd.concat(chunk_list)
    h_data.append(h_df)

./data/PUMS/psam_husa.csv
./data/PUMS/psam_husb.csv
./data/PUMS/psam_husc.csv
./data/PUMS/psam_husd.csv


In [18]:
h_data = pd.concat(h_data)
h_data

,SERIALNO,PUMA,ST,WGTP,NP,VEH
1259719,2015000007254,4953001,49,33,2,3
1259790,2015000017602,4953001,49,40,2,2
1259827,2015000022073,4953001,49,19,2,1
1259855,2015000024715,4953001,49,12,6,2
1259861,2015000025595,4953001,49,31,2,2
...,...,...,...,...,...,...
1917154,2019HU1404957,5600400,56,68,2,1
1917156,2019HU1406401,5600300,56,26,6,5
1917158,2019HU1406794,5600400,56,10,1,0
1917161,2019HU1408608,5600400,56,24,1,0


In [19]:
# Check for NA values
h_data.isna().sum()

SERIALNO    0
PUMA        0
ST          0
WGTP        0
NP          0
VEH         0
dtype: int64

In [9]:
# Remove records that are unable to be joined
z = pd.merge(h_data, p_data, on='SERIALNO')
serials_to_keep = list(z['SERIALNO'])
h_data = h_data[h_data['SERIALNO'].isin(serials_to_keep)]
p_data = p_data[p_data['SERIALNO'].isin(serials_to_keep)]
print(len(h_data))
print(len(p_data))

319350
761538


In [10]:
# Reset household ids
h_data.reset_index(inplace=True)
h_data.reset_index(inplace=True)
h_data['index'] = h_data['level_0']
h_data.drop(labels=['level_0'], axis=1)
p_data = pd.merge(h_data[['index','SERIALNO']], p_data, on='SERIALNO')

In [11]:
# Check for data issues
print(h_data.duplicated('index').any())

False


In [12]:
# Save seed data to file
h_data.to_csv('./populationsim-master/example_msa/data/seed_households.csv', index=False)
p_data.to_csv('./populationsim-master/example_msa/data/seed_persons.csv', index=False)

## Control Totals from ACS

#### MSA

In [3]:
DP03_cols = ['GEO_ID',
            'DP03_0047E',
            'DP03_0048E',
            'DP03_0049E',
            'DP03_0050E']
DP03_colnames = ['GEO_ID',
                'COW1-2',
                'COW3-5',
                'COW6-7',
                'COW8']
DP03 = pd.read_csv('./data/ACS/MSA/DP03.csv', low_memory=False, usecols=DP03_cols)
DP03.columns = DP03_colnames

In [4]:
DP03

,GEO_ID,COW1-2,COW3-5,COW6-7,COW8
0,id,Estimate!!CLASS OF WORKER!!Civilian employed p...,Estimate!!CLASS OF WORKER!!Civilian employed p...,Estimate!!CLASS OF WORKER!!Civilian employed p...,Estimate!!CLASS OF WORKER!!Civilian employed p...
1,310M500US10100,17901,2750,2089,123
2,310M500US10140,19655,6885,1693,90
3,310M500US10180,57562,11820,4692,212
4,310M500US10220,11653,4832,852,127
...,...,...,...,...,...
934,310M500US49660,199098,28474,13411,321
935,310M500US49700,50432,12962,5216,256
936,310M500US49740,56673,15964,3044,222
937,310M500US49780,31827,4319,2152,33


In [5]:
S1501_cols = ['GEO_ID',
             'S1501_C01_007E',
             'S1501_C01_008E',
             'S1501_C01_009E',
             'S1501_C01_010E',
             'S1501_C01_011E',
             'S1501_C01_012E',
             'S1501_C01_013E']
S1501_colnames = ['GEO_ID',
                 'SCHL1-11',
                 'SCHL12-15',
                 'SCHL16-17',
                 'SCHL18-19',
                 'SCHL20',
                 'SCHL21',
                 'SCHL22-24']
S1501 = pd.read_csv('./data/ACS/MSA/S1501.csv', low_memory=False, usecols=S1501_cols)
S1501.columns = S1501_colnames

In [6]:
S0802_cols = ['GEO_ID',
             'S0802_C01_001E',
             'S0802_C01_002E',
             'S0802_C01_003E',
             'S0802_C01_004E',
             'S0802_C01_005E',
             'S0802_C01_006E',
             'S0802_C01_007E',
             'S0802_C01_009E',
             'S0802_C01_010E',
             'S0802_C01_012E',
             'S0802_C01_013E',
             'S0802_C01_014E',
             'S0802_C01_015E',
             'S0802_C01_016E',
             'S0802_C01_017E',
             'S0802_C01_018E',
             'S0802_C01_029E',
             'S0802_C01_030E',
             'S0802_C01_031E',
             'S0802_C01_032E',
             'S0802_C01_033E',
             'S0802_C01_034E',
             'S0802_C01_035E',
             'S0802_C01_036E']
S0802_colnames = ['GEO_ID',
                 'TOTAL_P',
                 'AGEP16-19',
                 'AGEP20-24',
                 'AGEP25-44',
                 'AGEP45-54',
                 'AGEP55-59',
                 'AGEP60+',
                 'SEXM',
                 'SEXF',
                 'RAC1P1',
                 'RAC1P2',
                 'RAC1P3-5',
                 'RAC1P6',
                 'RAC1P7',
                 'RAC1P8',
                 'RAC1P9',
                 'PINCP-9999',
                 'PINCP10000-14999',
                 'PINCP15000-24999',
                 'PINCP25000-34999',
                 'PINCP35000-49999',
                 'PINCP50000-64999',
                 'PINCP65000-74999',
                 'PINCP75000+']
S0802 = pd.read_csv('./data/ACS/MSA/S0802.csv', low_memory=False, usecols=S0802_cols)
S0802.columns = S0802_colnames

# All S0802 counts are in percentage, must multiply by the total to get counts
pcts = S0802.iloc[1:,1:].astype(float)
pcts = (pcts / 100).multiply(pcts['TOTAL_P'], axis="index").astype(int)
S0802.iloc[1:,2:] = pcts.iloc[:,1:]

In [7]:
B08201_cols = ['GEO_ID',
              'B08201_001E',
              'B08201_002E',
              'B08201_003E',
              'B08201_004E',
              'B08201_005E',
              'B08201_006E',
              'B08201_007E',
              'B08201_013E',
              'B08201_019E',
              'B08201_025E']
B08201_colnames = ['GEO_ID',
                  'TOTAL_HH',
                  'VEH0',
                  'VEH1',
                  'VEH2',
                  'VEH3',
                  'VEH4+',
                  'NP1',
                  'NP2',
                  'NP3',
                  'NP4+']
B08201 = pd.read_csv('./data/ACS/MSA/B08201.csv', low_memory=False, usecols=B08201_cols)
B08201.columns = B08201_colnames

In [8]:
# Join data from different census tables
acs_dataframes = [DP03, S1501, S0802, B08201]
acs_data = functools.reduce(lambda left,right: pd.merge(left,right,on='GEO_ID'), acs_dataframes)

# Get MSA id from the extended geo id, remove acs labels, save marginal counts for popsim
acs_data.dropna(inplace=True)
acs_data = acs_data.iloc[1:,:]
acs_data['MSA'] = acs_data['GEO_ID'].str.slice(-5,).astype(int)
acs_data = acs_data[acs_data['MSA'].isin(msa_list)]
acs_data.to_csv('./populationsim-master/example_msa/data/control_totals_msa.csv', index=False)
acs_data

,GEO_ID,COW1-2,COW3-5,COW6-7,COW8,SCHL1-11,SCHL12-15,SCHL16-17,SCHL18-19,SCHL20,...,VEH0,VEH1,VEH2,VEH3,VEH4+,NP1,NP2,NP3,NP4+,MSA
34,310M500US11540,109654,12525,5574,237,3291,6074,50473,31576,20962,...,3796,27289,40996,14982,6392,24819,33614,14020,21002,11540
76,310M500US13220,33729,8615,1606,18,4175,8485,35369,16881,5805,...,4465,17092,18375,5961,2751,13745,17857,8209,8833,13220
80,310M500US13380,82255,17604,7192,129,3453,7044,34175,35153,16333,...,6039,24172,32980,15377,7955,23289,33313,13125,16796,13380
94,310M500US13980,53154,20609,3268,107,2981,6098,28201,20271,8407,...,3546,18313,23735,10904,6203,18278,23816,9596,11011,13980
113,310M500US14740,75914,32000,7092,231,2958,6706,40956,53017,19883,...,5181,28210,41352,19672,9498,26074,40476,15673,21690,14740
146,310M500US16220,33150,5664,2103,184,777,3319,16492,14812,6574,...,1277,8821,13026,6267,3408,9860,12171,4171,6597,16220
156,310M500US16620,79941,21362,4273,180,8208,16667,76323,34823,13181,...,11823,40821,38847,13487,5840,34601,40645,16516,19056,16620
160,310M500US16820,75016,24768,7322,308,5176,8495,28935,25811,8061,...,4762,26427,31760,13885,7169,24128,31761,12073,16041,16820
162,310M500US16940,32695,11917,2419,127,1220,3441,17162,17384,8403,...,2021,11249,14386,8080,3947,11234,14147,6014,8288,16940
251,310M500US20740,72185,10690,5145,193,2673,4476,32393,23249,17245,...,3529,20103,26260,10962,5728,19645,24762,9140,13035,20740


#### Tract

In [9]:
DP03_cols = ['GEO_ID',
            'DP03_0047E',
            'DP03_0048E',
            'DP03_0049E',
            'DP03_0050E']
DP03_colnames = ['GEO_ID',
                'COW1-2',
                'COW3-5',
                'COW6-7',
                'COW8']
DP03 = pd.read_csv('./data/ACS/TRACT/DP03.csv', low_memory=False, usecols=DP03_cols)
DP03.columns = DP03_colnames

In [10]:
S1501_cols = ['GEO_ID',
             'S1501_C01_007E',
             'S1501_C01_008E',
             'S1501_C01_009E',
             'S1501_C01_010E',
             'S1501_C01_011E',
             'S1501_C01_012E',
             'S1501_C01_013E']
S1501_colnames = ['GEO_ID',
                 'SCHL1-11',
                 'SCHL12-15',
                 'SCHL16-17',
                 'SCHL18-19',
                 'SCHL20',
                 'SCHL21',
                 'SCHL22-24']
S1501 = pd.read_csv('./data/ACS/TRACT/S1501.csv', low_memory=False, usecols=S1501_cols)
S1501.columns = S1501_colnames

In [11]:
S0802_cols = ['GEO_ID',
             'S0802_C01_001E',
             'S0802_C01_002E',
             'S0802_C01_003E',
             'S0802_C01_004E',
             'S0802_C01_005E',
             'S0802_C01_006E',
             'S0802_C01_007E',
             'S0802_C01_009E',
             'S0802_C01_010E',
             'S0802_C01_012E',
             'S0802_C01_013E',
             'S0802_C01_014E',
             'S0802_C01_015E',
             'S0802_C01_016E',
             'S0802_C01_017E',
             'S0802_C01_018E',
             'S0802_C01_029E',
             'S0802_C01_030E',
             'S0802_C01_031E',
             'S0802_C01_032E',
             'S0802_C01_033E',
             'S0802_C01_034E',
             'S0802_C01_035E',
             'S0802_C01_036E']
S0802_colnames = ['GEO_ID',
                 'TOTAL_P',
                 'AGEP16-19',
                 'AGEP20-24',
                 'AGEP25-44',
                 'AGEP45-54',
                 'AGEP55-59',
                 'AGEP60+',
                 'SEXM',
                 'SEXF',
                 'RAC1P1',
                 'RAC1P2',
                 'RAC1P3-5',
                 'RAC1P6',
                 'RAC1P7',
                 'RAC1P8',
                 'RAC1P9',
                 'PINCP-9999',
                 'PINCP10000-14999',
                 'PINCP15000-24999',
                 'PINCP25000-34999',
                 'PINCP35000-49999',
                 'PINCP50000-64999',
                 'PINCP65000-74999',
                 'PINCP75000+']
S0802 = pd.read_csv('./data/ACS/TRACT/S0802.csv', low_memory=False, usecols=S0802_cols)
S0802.columns = S0802_colnames

# All S0802 counts are in percentage, must multiply by the total to get counts
pcts = S0802.iloc[1:,1:]
pcts = pcts.replace('-','0', regex=True).astype(float)
pcts = (pcts / 100).multiply(pcts['TOTAL_P'], axis="index").astype(int)
S0802.iloc[1:,2:] = pcts.iloc[:,1:]

In [12]:
B08201_cols = ['GEO_ID',
              'B08201_001E',
              'B08201_002E',
              'B08201_003E',
              'B08201_004E',
              'B08201_005E',
              'B08201_006E',
              'B08201_007E',
              'B08201_013E',
              'B08201_019E',
              'B08201_025E']
B08201_colnames = ['GEO_ID',
                  'TOTAL_HH',
                  'VEH0',
                  'VEH1',
                  'VEH2',
                  'VEH3',
                  'VEH4+',
                  'NP1',
                  'NP2',
                  'NP3',
                  'NP4+']
B08201 = pd.read_csv('./data/ACS/TRACT/B08201.csv', low_memory=False, usecols=B08201_cols)
B08201.columns = B08201_colnames

In [13]:
# Join data from different census tables
acs_dataframes = [DP03, S1501, S0802, B08201]
acs_data = functools.reduce(lambda left,right: pd.merge(left,right,on='GEO_ID'), acs_dataframes)

# Get MSA id from the extended geo id, remove acs labels, save marginal counts for popsim
acs_data.dropna(inplace=True)
acs_data = acs_data.iloc[1:,:]
acs_data['TRACT'] = acs_data['GEO_ID'].str.slice(-11,).astype(int)
acs_data = acs_data[acs_data['TRACT'].isin(tract_list)]
acs_data.to_csv('./populationsim-master/example_msa/data/control_totals_tract.csv', index=False)
acs_data

,GEO_ID,COW1-2,COW3-5,COW6-7,COW8,SCHL1-11,SCHL12-15,SCHL16-17,SCHL18-19,SCHL20,...,VEH0,VEH1,VEH2,VEH3,VEH4+,NP1,NP2,NP3,NP4+,TRACT
68356,1400000US49053270100,2301,450,334,9,294,439,1262,1650,574,...,31,616,1083,617,425,548,1201,288,735,49053270100
68357,1400000US49053270200,774,191,96,7,11,97,592,605,178,...,7,258,402,321,99,259,463,127,238,49053270200
68358,1400000US49053270300,3358,722,158,19,152,258,1516,1707,881,...,131,902,1376,732,358,752,1600,355,792,49053270300
68359,1400000US49053270400,2609,413,365,0,27,280,1439,1490,734,...,16,958,1553,544,281,760,1610,259,723,49053270400
68360,1400000US49053270500,2580,645,321,0,32,179,1125,1121,910,...,28,380,1444,548,298,161,1179,418,940,49053270500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73957,1400000US56025001401,2632,188,135,0,119,323,1378,1084,531,...,25,317,761,512,532,494,698,282,673,56025001401
73958,1400000US56025001602,3059,432,334,29,241,351,1177,1174,426,...,0,660,1309,558,328,640,1115,446,654,56025001602
73959,1400000US56025001603,1412,429,129,7,0,83,552,614,246,...,0,220,375,428,284,186,616,209,296,56025001603
73960,1400000US56025001700,2008,355,86,0,15,162,1005,934,355,...,33,287,881,409,310,364,867,215,474,56025001700


#### Region

In [14]:
# Sum the values across all census tracts to get region marginals
region_data = acs_data.iloc[:,1:-1].apply(pd.to_numeric, errors='coerce')
region_data = pd.DataFrame(region_data.sum()).transpose()
region_data['REGION'] = '1'
region_data.dropna(inplace=True)
region_data.to_csv('./populationsim-master/example_msa/data/control_totals_region.csv', index=False)
region_data

,COW1-2,COW3-5,COW6-7,COW8,SCHL1-11,SCHL12-15,SCHL16-17,SCHL18-19,SCHL20,SCHL21,...,VEH0,VEH1,VEH2,VEH3,VEH4+,NP1,NP2,NP3,NP4+,REGION
0,5066730,938717,322000,10974,298194,464191,2179721,1845642,834038,1923769,...,357802,1544662,1896750,779645,393064,1408633,1767907,752311,1043072,1
